In [1]:
import scipy.stats as spt
import numpy as np
import pandas as pd
import seaborn as sb
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.stats.weightstats as sw
import matplotlib.pyplot as plt

In [2]:
# Two sample t test

fres = np.array([3.7,4.3,2.5,3.3,3.6,3.1])
soph = np.array([1.8,4.2,4.1,2.2,3.2,3.8])
twosam = spt.ttest_ind(fres,soph)
print(twosam)

print('t statistic = %.3f, p-value = %.3f'%(twosam))


print('1학년 :', np.mean(fres))
print('2학년 :', np.mean(soph))

Ttest_indResult(statistic=0.41473113113479565, pvalue=0.6870956353065248)
t statistic = 0.415, p-value = 0.687
1학년 : 3.4166666666666674
2학년 : 3.216666666666667


In [3]:
# Dataframe

data1=pd.DataFrame(data=np.array([[3.7,1.8,3.3,4.1],[4.3,4.2,3.7,3.8],[2.5,4.1,3.4,3.5],[3.3,2.2,3.9,3.2],[3.6,3.2,np.nan,2.3]]),
                   columns=['fre','sop','jun','sen'])
print(data1['jun'],'\n')

sop = data1['sop']
jun = data1['jun']
jun = jun.fillna(jun.mean())

print(jun,'\n')

tsam = spt.ttest_ind(sop,jun)
print(tsam)

0    3.3
1    3.7
2    3.4
3    3.9
4    NaN
Name: jun, dtype: float64 

0    3.300
1    3.700
2    3.400
3    3.900
4    3.575
Name: jun, dtype: float64 

Ttest_indResult(statistic=-0.9550271234200176, pvalue=0.36752757550294646)


In [4]:
# Paired t test

before=np.array([68,61,60,68,67,64,66,67,66,67,72,74,61,71,58,77])
after=np.array([56,55,67,62,59,67,50,60,59,53,60,65,62,61,64,57])

pairsam = spt.ttest_rel(before,after)
print(pairsam)

Ttest_relResult(statistic=3.562626515972126, pvalue=0.002834843356363954)


In [5]:
# Statsmodels를 이용한 t test

fre=data1['fre']
sen=data1['sen']


# 가설에는 two-sided / larger / smaller 3가지, 분산에는 pooled, unequal 2가지, value는 두 집단의 평균 차이가 얼마인지 나타내는 수
tsams=sw.ttest_ind(fre,sen,alternative='two-sided',usevar='pooled',value=0)
print('tstat = \t',tsams[0],'\np-val = \t',tsams[1],'\ndegree of freedom = \t',tsams[2])



# 쌍체검정 (One-sided)

ptsams = sw.ttost_paired(before,after,0.1,0.2)
print(ptsams)

tstat = 	 0.23453251491009644 
p-val = 	 0.8204627528467475 
degree of freedom = 	 8.0
(0.9982467120464241, (3.510806493921623, 0.0015764190017186505, 15.0), (3.458986471871119, 0.9982467120464241, 15.0))


In [6]:
# Statsmodels를 이용한 t test

fre=data1['fre']
sen=data1['sen']


# 가설에는 two-sided / larger / smaller 3가지, 분산에는 pooled, unequal 2가지, value는 두 집단의 평균 차이가 얼마인지 나타내는 수
tsams=sw.ttest_ind(fre,sen,alternative='two-sided',usevar='pooled',value=0)
print('tstat = \t',tsams[0],'\np-val = \t',tsams[1],'\ndegree of freedom = \t',tsams[2])



# 쌍체검정 (One-sided)

ptsams = sw.ttost_paired(before,after,0.1,0.2)
print(ptsams)

tstat = 	 0.23453251491009644 
p-val = 	 0.8204627528467475 
degree of freedom = 	 8.0
(0.9982467120464241, (3.510806493921623, 0.0015764190017186505, 15.0), (3.458986471871119, 0.9982467120464241, 15.0))


In [8]:
# Excel file read
df = pd.read_csv('score1.csv')
print(df)

print(df.grade.unique())

model = ols('score ~ C(grade)', df).fit()
anova_lm(model)

    score grade
0      75   fre
1      85   fre
2      34   fre
3      23   fre
4      99   sop
5      84   sop
6      17   sop
7      56   sop
8      43   jun
9      22   jun
10     88   jun
11     71   jun
12     97   sen
13     44   sen
14     82   sen
15     74   sen
['fre' 'sop' 'jun' 'sen']


,df,sum_sq,mean_sq,F,PR(>F)
C(grade),3.0,1000.25,333.416667,0.372967,0.774051
Residual,12.0,10727.50,893.958333,NaN,NaN


In [9]:
# One-way에서 scipy.stats.f_oneway() 를 사용할 때의 결측치 제거

# data[~np.isnan(data)]

In [10]:
# 오차의 가정 점검

print(spt.shapiro(df.score[df.grade=='fre']),'\n')

print(spt.levene(df.score[df.grade=='fre'],df.score[df.grade=='sop'],df.score[df.grade=='jun'],df.score[df.grade=='sen']),'\n')

print(spt.bartlett(df.score[df.grade=='fre'],df.score[df.grade=='sop'],df.score[df.grade=='jun'],df.score[df.grade=='sen']))


ShapiroResult(statistic=0.8843879699707031, pvalue=0.3577159345149994) 

LeveneResult(statistic=0.6665090995036634, pvalue=0.5885604746273037) 

BartlettResult(statistic=0.5881874396406543, pvalue=0.8991311844728094)


In [11]:
# 사후 검정

from statsmodels.sandbox.stats.multicomp import MultiComparison
from statsmodels.stats.multicomp import pairwise_tukeyhsd

comp = MultiComparison(df.score, df.grade)


# Bonferroni
opt = comp.allpairtest(spt.ttest_ind, method='bonf')
print(opt[0])

# Tukey's HSD
hsd = pairwise_tukeyhsd(df['score'], df['grade'], alpha=0.05)
hsd.summary()

Test Multiple Comparison ttest_ind 
FWER=0.05 method=bonf
alphacSidak=0.01, alphacBonf=0.008
group1 group2   stat   pval  pval_corr reject
---------------------------------------------
   fre    jun  -0.083 0.9366       1.0  False
   fre    sen -1.0621 0.3291       1.0  False
   fre    sop -0.4138 0.6934       1.0  False
   jun    sen -0.9914 0.3598       1.0  False
   jun    sop -0.3445 0.7422       1.0  False
   sen    sop  0.4836 0.6458       1.0  False
---------------------------------------------


group1,group2,meandiff,p-adj,lower,upper,reject
fre,jun,1.75,0.9,-61.0255,64.5255,False
fre,sen,20.0,0.7635,-42.7755,82.7755,False
fre,sop,9.75,0.9,-53.0255,72.5255,False
jun,sen,18.25,0.8076,-44.5255,81.0255,False
jun,sop,8.0,0.9,-54.7755,70.7755,False
sen,sop,-10.25,0.9,-73.0255,52.5255,False


In [12]:
# Two-way ANOVA

data2 = pd.read_csv('score2.csv')
data2.head()

,score,grade,class
0,75,fre,A
1,85,fre,B
2,34,fre,A
3,23,fre,A
4,99,sop,C


In [13]:
#data2.groupby('grade').agg(len)
data2.groupby('class').agg(len)
#data2.groupby(['grade', 'class']).agg(len)

,score,grade
class,,
A,4,4
B,4,4
C,4,4
D,4,4


In [17]:
model2 = ols('score ~ C(grade)*C(class)', data=data2).fit()
anova_lm(model2)

SyntaxError: invalid syntax (<unknown>, line 1)

In [15]:
# statsmodels anova_lm type : Referred https://jooskorstanje.com/anova-types-of-sums-of-squares-notebook.html

weekday = ['sat', 'sat', 'sat', 'sat', 'sat', 'sat', 'sun', 'sun', 'sun', 'sun']
weather = ['rain', 'rain', 'rain', 'rain', 'rain', 'sun', 'sun', 'sun', 'sun', 'sun']
sales = [100, 100, 100, 100, 100, 10000, 10000, 10000, 10000, 10000]

data = pd.DataFrame({'weekday': weekday, 'weather': weather, 'sales': sales})
data

,weekday,weather,sales
0,sat,rain,100
1,sat,rain,100
2,sat,rain,100
3,sat,rain,100
4,sat,rain,100
5,sat,sun,10000
6,sun,sun,10000
7,sun,sun,10000
8,sun,sun,10000
9,sun,sun,10000


In [16]:
# Type I tells us that weekday is more important. The interaction effect is not signifcant.
lm = ols('sales ~ C(weekday)*C(weather)',data=data).fit()
table = sm.stats.anova_lm(lm, typ=1) # Type 1 ANOVA DataFrame
print(table)

                        df        sum_sq       mean_sq             F  \
C(weekday)             1.0  1.633500e+08  1.633500e+08  1.684720e+31   
C(weather)             1.0  8.167500e+07  8.167500e+07  8.423598e+30   
C(weekday):C(weather)  1.0  1.464446e-24  1.464446e-24  1.510365e-01   
Residual               7.0  6.787183e-23  9.695975e-24           NaN   

                              PR(>F)  
C(weekday)             1.345639e-107  
C(weather)             1.522416e-106  
C(weekday):C(weather)   7.090969e-01  
Residual                         NaN  


In [18]:
# Type II tells us that weather is more important. There is no interaction effect.
lm = ols('sales ~ C(weekday) + C(weather)',data=data).fit()
table = sm.stats.anova_lm(lm, typ=2) # Type 2 ANOVA DataFrame
print(table)

                  sum_sq   df             F         PR(>F)
C(weekday)  3.242548e-23  1.0  3.949889e-01   5.496373e-01
C(weather)  8.167500e+07  1.0  9.949187e+29  2.688553e-103
Residual    5.746450e-22  7.0           NaN            NaN


In [19]:
# Type III tells us that weekday is more important. The interaction effect is not signifcant.
lm = ols('sales ~ C(weekday)*C(weather)',data=data).fit()
table = sm.stats.anova_lm(lm, typ=3) # Type 3 ANOVA DataFrame
print(table)

                             sum_sq   df             F         PR(>F)
Intercept              5.000000e+04  1.0  5.156779e+27   2.681948e-95
C(weekday)             2.382280e-23  1.0  2.456978e+00   1.609890e-01
C(weather)             8.167500e+07  1.0  8.423598e+30  1.522416e-106
C(weekday):C(weather)  4.235165e-23  1.0  4.367962e+00   7.497369e-02
Residual               6.787183e-23  7.0           NaN            NaN
